In [25]:
%pip install pdfplumber rouge-score openai -q

Note: you may need to restart the kernel to use updated packages.


# 📝 Student Summary Evaluator

An intelligent, AI-powered tool that evaluates a student's summary using a **hybrid ground-truth system** — combining an AI-generated reference summary *and* key points extracted directly from the lecture — across **ten dimensions**.

### 🔀 Hybrid Evaluation Strategy

| Source | Used For |
|---|---|
| **AI Reference Summary** | Style/structure comparison (Correctness, Coherence, Conciseness) |
| **Lecture Key Points** *(extracted directly from lecture)* | Ground-truth coverage (Completeness, Missing Key Points) |
| **Raw Lecture Text** | Relevance, Hallucination, Factual Accuracy, Terminology |
| **Student Summary alone** | Critical Analysis |

> ⚡ **Why hybrid?** If the AI reference summary is vague or incomplete, any metric that compares *only* against it becomes unreliable. Lecture key points are extracted verbatim from the source text and act as an independent, uncorruptible ground truth.

### 📊 Evaluation Dimensions

| # | Metric | Ground Truth Used |
|---|---|---|
| 1 | **Correctness** | Hybrid: 40% vs lecture · 35% vs reference · 25% ROUGE |
| 2 | **Relevance** | Raw lecture paragraphs |
| 3 | **Coherence** | Student summary (AI-judged) |
| 4 | **Completeness** | Hybrid: 50% lecture key points · 50% ROUGE vs reference |
| 5 | **Conciseness** | Reference summary length |
| 6 | **Terminology** | Raw lecture text |
| 7 | **Hallucination** | Raw lecture text (AI-verified) |
| 8 | **Missing Key Points** | Lecture key points (independent of reference summary) |
| 9 | **Factual Accuracy** | Raw lecture text (AI-verified) |
| 10 | **Critical Analysis** | Student summary (AI-judged) |

> **Workflow:** Load lecture → Extract key points + generate reference summary → Paste student summary → Get scores


In [26]:
import os, re, json, warnings
import numpy as np
import pdfplumber
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from IPython.display import display, HTML

warnings.filterwarnings("ignore")
nltk.download("punkt",     quiet=True)
nltk.download("punkt_tab", quiet=True)
nltk.download("stopwords", quiet=True)

print("✅  Libraries loaded.")


✅  Libraries loaded.


## ⚙️ Step 1 — Configuration

Paste your **OpenRouter API key** below (`sk-or-v1-...`).  
Get one at 👉 [https://openrouter.ai/keys](https://openrouter.ai/keys)


In [27]:
# ─── PASTE YOUR OPENROUTER API KEY HERE ───────────────────────────────────────
OPENAI_API_KEY = "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"   # ← paste sk-or-v1-... here
OPENAI_MODEL   = "openai/gpt-4o-mini"         # gpt-4o-mini via OpenRouter

# OpenRouter uses the same OpenAI SDK — just point to a different base URL
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1",
)

# Quick connectivity check
_test = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{"role": "user", "content": "Reply with the word OK only."}],
    max_tokens=5,
)
print(f"✅  '{OPENAI_MODEL}' via OpenRouter — ping: {_test.choices[0].message.content.strip()}")


✅  'openai/gpt-4o-mini' via OpenRouter — ping: OK


## 📄 Step 2 — Load the Lecture

Provide the lecture using **one** of these options:
- **Option A** — Paste the lecture text directly into `LECTURE`
- **Option B** — Set `LECTURE` to a `.pdf` file path, e.g.  
  `LECTURE = load_lecture(r"C:\path\to\lecture.pdf")`


In [28]:
def load_lecture(source: str) -> str:
    """Load lecture from a plain-text string or a .pdf file path."""
    source = source.strip()
    if source.lower().endswith(".pdf"):
        text = ""
        with pdfplumber.open(source) as pdf:
            for page in pdf.pages:
                t = page.extract_text()
                if t:
                    text += t + "\n"
        return text.strip()
    return source


# ─── OPTION A – paste lecture text directly ───────────────────────────────────
LECTURE = """Introduction: 
Drone warfare has become a defining aspect of modern combat, raising profound ethical 
questions in engineering. Small unmanned aerial vehicles (UAVs) now conduct surveillance 
and precision strikes that were once the sole domain of manned aircraft. Their availability and 
effectiveness have significantly altered conflict: analysts note that both sides in Ukraine have 
deployed thousands of small drones for intelligence, reconnaissance, and direct attack . The 
war in Ukraine thus exemplifies the new era of drone warfare and its ethical implications. 
Engineers designing these systems face dilemmas such as balancing mission autonomy against 
accountability, and optimizing cost-efficiency at the risk of indiscriminate use.  
The importance of this topic to engineering ethics lies in the tension between professional 
duties and wartime imperatives. Codes of ethics (e.g. the NSPE Code) insist engineers “hold 
paramount the safety, health, and welfare of the public” , yet military drones are explicitly built 
to kill. This study will explore how engineering decisions in drone design and deployment 
create moral hazards, and examine specific case studies from the Russia-Ukraine conflict. 
1 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
The objectives are to:  
• outline the technical and ethical trade-offs in modern combat drones 
• Compare different drone models in terms of  design and use 
• analyze documented drone strikes in Ukraine and their humanitarian effects 
• reflect on the broader responsibilities of engineers in armed conflict.  
With added examples and deeper analysis, this paper seeks to provide a nuanced understanding 
of drones’ dual nature as tools of progress and destruction. 
Technical and Ethical Dimensions of Drone Warfare: 
Design Trade-offs:Shahed-136 vs. Bayraktar TB2 
Combat drones vary widely in complexity, cost, and capability. A clear illustration is the 
difference between the Iranian-made Shahed-136 and the Turkish Bayraktar TB2. The Shahed
136 is a kamikaze “loitering munition” – a single-use drone carrying a warhead for one-way 
missions .It is small (length ≈3.5 m, wingspan ≈2.5 m, weight ~200 kg) with a crude autopilot 
(inertial/GPS guidance) and a ~50 kg explosive payload . In contrast, the Bayraktar TB2 is a 
much larger reusable UAV (length 6.5 m, span 12 m, MTOW 700 kg) with sophisticated 
avionics. It carries 4 smart guided bombs or missiles (total 150 kg payload) and can fly for 27 
hours with live human control at up to 222 km/h . (fig1)Relative sizes of the Iranian Shahed
136 (left) and the Turkish Bayraktar TB2 (right) drones. The Shahed’s small, delta-wing design 
and limited range (order 1,000–2,500 km) contrasts with the much larger, long-endurance TB2.                                 
Figure 1 
2 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
These design choices reflect different priorities. 
The Shahed is extremely cheap (reported 
production cost tens of thousands of USD) , 
allowing Russia to launch salvos of dozens each 
night. This swarming strategy saturates air defenses 
but sacrifices accuracy and reusability. By contrast, 
a Bayraktar costs on the order of $1–5 million , is 
human-piloted, and must return after each mission. 
Its advanced sensors and precision weapons enable 
targeted strikes with minimal collateral damage 
when used properly, but its high cost and 
vulnerability to air defenses limit the number that 
can be deployed. Engineers must trade cost vs. 
precision and autonomy vs. control. High autonomy 
(as in the Shahed’s simple AI pilot) can permit 
stand-off attacks without risking pilots but reduces 
accountability and can lead to more civilian hits if 
guidance fails. In both designs, reliability is also an 
ethical issue: a kamikaze drone that misses its target 
may crash indiscriminately, and a multi-ton UAV 
malfunction could fall on civilians. The necessity to 
protect public welfare (as per engineering ethics 
codes) clashes with the engineering goal to 
maximize a weapon’s effectiveness.(fig2) shows a 
comparison between Shahed-136 and Bayraktar 
TB2 
Control and Autonomy 
Drone 
Shahed 136 Bayraktar TB2 
Manufacturer 
HESA 
Baykar 
Length 
3,5 m 
6,5 m 
Wing span 
2,5 m 
12 m 
MOTW 200 kg* 
650 kg 
Speed 
185 km/h 
130 km/h 
Range 
2.500 km* 
150 km 
Engine 
50 hp 
110 hp 
Payload 
40 kg 
140 kg 
Takeoff 
By platform 
Runway 
Price 
$ 20,000 
$ 1,000,000 
Figure 2 
Modern drones often incorporate various levels of autonomy. A key question is how much decision 
making the drone can do independently. The Shahed-136, for example, uses a pre-programmed flight 
plan to loiter and descend on a target, with only a basic inertial/GPS guidance . The TB2, by contrast, 
is controlled in real-time by a pilot via encrypted datalink, allowing human judgment in target selection 
. The increasing integration of AI (e.g., for obstacle avoidance or automated target recognition) raises 
new ethical concerns: Who is responsible if an autonomous system misidentifies a civilian structure as 
a military target? The issue of accountability looms large – current engineering codes expect 
transparency and verification, but autonomous drones can obscure decision chains. Some defense 
3 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
analysts warn that technology often outpaces regulation, noting that “drones have become more capable, 
but ethical and regulatory frameworks have lagged” . Engineering professionals thus face the dilemma 
of innovating in warfare technology while ensuring human oversight and compliance with international 
law. 
Engineers’ Responsibilities and Codes of Ethics 
Engineers involved in military projects must reconcile their work with professional ethical standards. 
The NSPE Code of Ethics explicitly states that “Engineers shall hold paramount the safety, health, 
and welfare of the public.” . Designing lethal systems seems at odds with this canon, but engineers 
often justify weapons development under national security imperatives. However, even military 
engineers are bound to avoid deceptive or malicious acts and to refuse assignments that endanger public 
life without proper safeguards. For example, if an engineer learns that a drone design is likely to fail 
dangerously or breach the laws of armed conflict, the NSPE code would require them to raise concerns 
or withdraw from the project . Ethical guidelines from IEEE and IET similarly emphasize human 
oversight: weapons should not operate with full autonomy to kill without human judgment.  
In practice, some argue engineers cannot be held personally liable if a lawful weapon is misused. 
Nevertheless, ethical training encourages engineers to consider the downstream consequences of their 
designs. If a drone’s sensors are insufficient to distinguish combatants from civilians, engineers must 
recognize the risk of collateral damage. Engineering societies increasingly call for “caution” in weapon 
automation – noting that “drones pose serious ethical dilemmas around how, and whether, to 
regulate” their use . By applying professional codes, engineers must ensure their systems include fail
safe and respect targeting protocols (e.g., the Geneva Conventions’ principle of distinction), even when 
operating in war. This means rigorous testing, honest reporting, and possibly whistleblowing if safety 
is compromised. 
Civilian Impact and Broader Consequences 
The deployment of drones in Ukraine has had severe humanitarian consequences. Though proponents 
claim precision, in practice both Russian and Ukrainian drone strikes have hit civilian areas. For 
instance, in January 2025 the UN reported short-range drones killed more Ukrainians than any other 
weapon: 27% of civilians killed (38 out of 139) and 30% of injuries (223 out of 738) in that month were 
from drone strikes as shown in fig(3). These “FPV” drone bombs, launched by operators on the ground, 
frequently struck cars, streets, and other public places. The Head of the UN Human Rights Mission in 
Ukraine noted, “Short-range drones now pose one of the deadliest threats to civilians in frontline areas.”. 
4 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
Likewise, a UN report documented a single Russian drone strike on 17 May 2025 that killed 9 evacuees 
in a civilian minibus  
Figure 3 
Persistent drone barrages also inflict psychological trauma. Civilians describe almost nightly air-raid 
sirens as “bombing of sleep” – people cannot rest, leading to widespread insomnia and PTSD risks . 
Mental health professionals in Ukraine warn that sleep deprivation from constant drone attacks 
“weakens immune systems and raises the risk of long-term illnesses” . For example, one journalist notes 
Ukrainians have developed hypersensitivity: ordinary noises trigger panic, and many now live in 
“feigned arousal” mode . A 2024 study found that regions exposed to frequent air alarms (rockets and 
drones) showed significantly higher PTSD and sleep disturbance than quieter areas . In effect, cheap 
drones do not just destroy structures; they erode morale and well-being far from the front lines.  
Infrastructure damage is another ethical concern. Reports from Ukraine’s civil defense indicate that 
drones have struck schools, power plants, and apartment blocks. For instance, the April 2025 drone 
volley on Kyiv shattered windows of municipal buildings, injuring dozens and damaging civilian 
property . In Kherson city (partially occupied), a January 2025 strike on a passenger bus killed a man 
and injured nine , and local officials note such incidents occur “almost daily” . President Zelenskiy 
publicly called the April 2025 Marhanets bus hit (9 killed) “a deliberate war crime” . These episodes 
show that drone operators – and by extension the engineers who enable them – have a responsibility to 
consider foreseeable civilian harm under just-war norms. Collateral damage from falling drone 
5 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
wreckage (especially when thousands of pounds of debris rain down) can be severe and is largely 
unavoidable with loitering munitions. 
Ethical Implications for Society and Future Warfare  
Widespread drones also affects public trust in technology and warfare. When people see autonomous 
or remotely piloted weapons causing civilian suffering, they may lose faith in institutions and engineers 
who develop such systems. Communities report distrust not only of the warring parties but of any 
external actors promoting drone technology. Moreover, the success of drones in Ukraine has spurred a 
global arms race: dozens of countries are now rapidly expanding their drone fleets. Ethically, this raises 
the prospect of future conflicts being fought by increasingly automated means – a trend that demands 
robust oversight. Analysts warn that without new norms, the convenience of drone strikes could lower 
the bar for entering conflicts . Indeed, one dilemma is whether removing pilot risk makes leaders more 
prone to launch attacks. If the human cost to one’s own forces is negligible, will governments bypass 
diplomacy more readily? Engineering ethicists worry that “the perceived safety for operators lowers the 
threshold for war,” potentially leading to more frequent or prolonged conflicts . 
The combination of low cost and high lethality of drone swarms is creating a new paradigm. Combatants 
and civilians alike see their world transformed by night-time drone swarms. Society must ask: should 
restrictions be placed on fully autonomous swarms? Should there be an international treaty limiting 
kamikaze drones? Engineers, who often champion innovation, must also engage in public and policy 
dialogues to help shape such regulations in line with humanitarian principles. In the meantime, they 
must embed ethical safeguards in design (e.g., programming strict no-fire zones, implementing human
in-the-loop controls, and ensuring transparency in target selection algorithms) to uphold the 
profession’s duty to the public. 
Case Study: Drone Operations in the Russia Ukraine War 
This section examines specific drone attacks during the current conflict and the ethical issues they 
illustrate. 
Kyiv and Major Cities: On 24 April 2025, Russia launched a massive, combined missile-and drone 
assault on Kyiv. U.S. officials reported “at least 12 people” killed and about 90 wounded . Missiles 
destroyed buildings, but several deaths were caused by drones striking near civilian areas. Emergency 
services combed rubble for hours, illustrating how even precision tools failed to avoid urban casualties. 
The scale of this attack – the largest on Kyiv in 2025 – drew rare international condemnation. U.S. 
President Trump himself warned Russia to “STOP” after hearing of “not necessary” strikes on civilians 
6 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
. 
This incident underscores the ethical question of proportionality: were these drone strikes aimed 
exclusively at military infrastructure, or did they recklessly endanger non-combatants? 
Public Transport: Drones have repeatedly targeted buses. In Kherson city (January 2025), a Russian 
drone hit a civilian coach, killing a 49-year-old man and wounding nine . The governor reported such 
strikes occur “almost daily” in frontline regions. In Dnipropetrovsk region (April 2025), a Russian 
attack on the city of Marhanets struck a mining workers’ bus on a highway, killing nine and injuring 
nearly fifty . President Zelenskiy decried this as a war crime since the bus was an “ordinary civilian 
object” . Engineers who build targeting systems must ask: could improve sensor fidelity or stricter 
verification have prevented these tragedies? The likely answer is yes – human operators rely on 
intelligence cues. If engineers do not design robust friend-vs-foe identification, even an accurate 
weapon can become ethically compromised. 
Battlefield Surveillance: On the military side, Ukrainian forces have used Bayraktar TB2 drones as 
recon assets. These drones have scouted Russian troop positions and even struck isolated armored 
vehicles far behind enemy lines . For example, one open-source analysis credits small Turkish drones 
with destroying dozens of Russian tanks and supplies by providing real time targeting data . Ethically, 
surveillance drones entail fewer direct harms than attack drones, but still pose questions: do engineers 
ensure their cameras respect privacy when used in civilian areas? Should drone software be configured 
to disable live streaming when missiles are absent? While these issues have not been as prominent in 
wartime reporting, they reflect the same principle: engineers should integrate constraints to protect non
targeted people whenever possible. 5 In sum, the Ukraine case studies reveal that despite advanced 
guidance, many drone strikes still hit civilians. As seen in figure below, Russia increasingly relies on 
one-way attack drones to sustain its attack on Ukrainian critical infrastructure and political centers 
consistent with their concept of noncontact war and using long-range precision strike assets. Every 
night, these systems cause millions of Ukrainians to head to bomb shelters and mobile air 
Figure 4 
7 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
This highlights the engineer’s ethical duty to minimize harm. Even if a weapon functioned “as 
designed,” its deployment context can make it unethical. Therefore, engineers should anticipate misuse 
and error on the side of caution, advocating for strict adherence to the laws of armed conflict in any 
drone system they produce. 
Conclusion : 
Drone warfare in Ukraine exemplifies the complex ethical challenges of engineering modern weapons. 
This case study shows that design decisions – cost, autonomy, reliability – have moral weight. The 
Iranian Shahed-136 and Turkish Bayraktar TB2 represent two ends of the spectrum: one a cheap mass 
produced “suicide” UAV, the other an expensive precision drone. Both have strategic value, but their 
use raises engineers’ ethical responsibilities. To prevent civilian suffering, engineers must embed 
safeguards, maintain human oversight, and heed professional codes requiring public safety to be 
paramount . 
Key findings include: (1) Cost-effective drones like the Shahed enable mass attacks that overwhelm 
defenses but often violate the principles of distinction, resulting in high civilian casualties . (2) 
Precision drones like the TB2 can be more discriminating but are costly and vulnerable, potentially 
limiting their deployment. (3) Psychological harm from nightly drone barrages imposes long-term 
burdens on society . (4) Engineer ethics require balancing military objectives with minimization of 
harm; according to NSPE, engineers must prioritize public welfare, even when working on defense 
projects . 
Readers should take away that engineering in warfare is never neutral: every technical choice has social 
and moral implications. As autonomous weapons proliferate, engineers must engage proactively in 
ethical reflection and policymaking. Ensuring that drone technology serves defensive security goals 
without undermining human values will require strict professional vigilance. Ultimately, the future of 
autonomous warfare will depend on the engineering community’s commitment to ethical principles, 
just as much as on technical innovation. 
8 
Title: Drone Warfare in the Russia-Ukraine Conflict                         
Spring 2025 
References: 
1-NSPE, Code of Ethics for Engineers, National Society of Professional Engineers (2019), p.1-2 .  
2-HRMMU, In Ukraine Short-Range Drones Become Most Dangerous Weapon for Civilians, UN 
Human Rights Monitoring Mission in Ukraine, 11 Feb 2025 .  
3-UN Geneva, Civilians killed in drone strike in eastern Ukraine: UN rights monitors, 17 May 2025 .  
4-Reuters, Russian drone kills one, injures nine in passenger bus in Ukraine’s Kherson, 6 Jan 2025 . 
5-Reuters, Russian drone strike on bus kills 9 in Ukrainian city of Marhanets, Kyiv says, 23 Apr 2025.  
6-Reuters, Mass drone attack kills three, injures at least 30 in Ukraine’s Dnipro, governor says, 17 Apr 
2025 . 7-7-Reuters, Trump tells Putin to ‘STOP’ after Russian attack kills 12 in Kyiv, 24 Apr 2025 .  
8-Kyiv Independent, K. Khrebet et al., “We are exhausted” — how Ukrainians cope with nightly 
Russian drone attacks, 11 Jan 2024 .  
9-Australian Army Research Centre, How are Drones Changing Modern Warfare? (Land Power Forum 
article), Sept 2022 . 
10-Army Recognition, Shahed-136 loitering munition (Technical Data) (2024) .  
11-Bayraktar TB2 technical specifications, Wikipedia (accessed 2024) . 
12-Modern Diplomacy, B. Bansod, The Drone Wars: A New Era of Conflict, 17 Dec 2024 .  
13-CSIS, N. Hollenbeck et al., Calculating the Cost-Effectiveness of Russia’s Drone Strikes, Oct 2024.  
14-SAN News (via Getty Images), Sleepless in Ukraine: Drone strikes and the rise of antidepressant 
use, 25 Apr 2025 . 
9"""

# ─── OPTION B – provide a PDF file path (uncomment and edit) ──────────────────
# LECTURE = load_lecture(r"C:\Users\you\Documents\lecture.pdf")

LECTURE = load_lecture(LECTURE)
print(f"✅  Lecture loaded  ({len(LECTURE.split()):,} words).")


✅  Lecture loaded  (2,835 words).


## 🤖 Step 3 — Generate Reference Summary & Extract Lecture Key Points

Two things are prepared here in parallel:

1. **AI Reference Summary** — a well-written summary used for style/structure comparison  
2. **Lecture Key Points** — 15 short, concrete, factual checkpoints extracted *directly* from the lecture text (not from the summary). These serve as the **independent ground truth** for completeness and missing-point checks — so even a weak reference summary cannot corrupt those scores.


In [29]:
def openai_chat(prompt: str, max_tokens: int = 1500) -> str:
    """Send a prompt to OpenAI and return the response text."""
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are an expert academic assistant."},
            {"role": "user",   "content": prompt},
        ],
        max_tokens=max_tokens,
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()


# ── 1. Reference Summary ──────────────────────────────────────────────────────
def generate_reference_summary(lecture: str) -> str:
    """Ask OpenAI to produce a thorough reference summary of the lecture."""
    prompt = (
        "Read the lecture below carefully and write a comprehensive, accurate, "
        "and well-structured summary that:\n"
        "  • Covers ALL key concepts, definitions, and main ideas\n"
        "  • Preserves important terminology exactly as used in the lecture\n"
        "  • Is logically organised with clear transitions between ideas\n"
        "  • Is written in clear, formal academic prose\n\n"
        f"LECTURE:\n{lecture}\n\n"
        "SUMMARY:"
    )
    return openai_chat(prompt, max_tokens=1500)


# ── 2. Lecture Key Points — independent ground truth ─────────────────────────
def extract_key_points(lecture: str) -> list:
    """
    Ask the AI to extract 15 short, concrete, factual key points directly
    from the lecture text. These are used as ground truth for completeness
    and missing-point checks, independent of the reference summary quality.
    Returns a list of strings.
    """
    prompt = (
        "Read the lecture below and extract exactly 15 key points.\n\n"
        "Rules for key points:\n"
        "  • Each point must be a SHORT, CONCRETE, FACTUAL statement (1–2 sentences max)\n"
        "  • Cover the most important concepts, facts, arguments, trade-offs, and conclusions\n"
        "  • Include specific details where relevant (names, numbers, examples)\n"
        "  • Do NOT paraphrase vaguely — be precise and grounded in the lecture text\n"
        "  • Cover different sections of the lecture (not just the introduction)\n\n"
        f"LECTURE:\n{lecture}\n\n"
        'Return ONLY valid JSON: {"key_points": ["<point1>", "<point2>", ..., "<point15>"]}'
    )
    try:
        raw  = openai_chat(prompt, max_tokens=1000)
        m    = re.search(r"\{.*\}", raw, re.DOTALL)
        data = json.loads(m.group()) if m else {}
        pts  = data.get("key_points", [])
        return pts if pts else [lecture[:200]]   # fallback
    except Exception:
        return [lecture[:200]]


# ── Run both ──────────────────────────────────────────────────────────────────
print("⏳  Generating AI reference summary…")
REFERENCE_SUMMARY = generate_reference_summary(LECTURE)
print(f"✅  Reference summary ready  ({len(REFERENCE_SUMMARY.split()):,} words).\n")

print("⏳  Extracting lecture key points (ground truth)…")
LECTURE_KEY_POINTS = extract_key_points(LECTURE)
print(f"✅  {len(LECTURE_KEY_POINTS)} key points extracted.\n")

print("─" * 70)
print("REFERENCE SUMMARY:\n")
print(REFERENCE_SUMMARY)
print("\n" + "─" * 70)
print("LECTURE KEY POINTS (ground truth):\n")
for i, pt in enumerate(LECTURE_KEY_POINTS, 1):
    print(f"  {i:2}. {pt}")


⏳  Generating AI reference summary…
✅  Reference summary ready  (618 words).

⏳  Extracting lecture key points (ground truth)…
✅  15 key points extracted.

──────────────────────────────────────────────────────────────────────
REFERENCE SUMMARY:

The lecture on drone warfare, particularly in the context of the Russia-Ukraine conflict, delves into the ethical and technical complexities associated with the use of unmanned aerial vehicles (UAVs) in modern combat. It highlights how drones have transformed warfare, enabling surveillance and precision strikes that were traditionally conducted by manned aircraft. The ongoing conflict in Ukraine serves as a case study to explore the implications of drone warfare, where both sides have deployed thousands of small drones for intelligence, reconnaissance, and direct attacks. This shift raises significant ethical dilemmas for engineers who design these systems, particularly regarding the balance between mission autonomy and accountability, as well

## ✍️ Step 4 — Enter the Student Summary

Paste the student's summary into `STUDENT_SUMMARY` below.


In [ ]:
# ─── PASTE THE STUDENT SUMMARY HERE ──────────────────────────────────────────
#STUDENT_SUMMARY = """This paper talks about drones and how they are used in the Russia-Ukraine war, but mostly it says drones are important because they are modern technology. It mentions some drones like Iranian and Turkish ones, but does not really explain them clearly. The paper also talks about ethics, but only in a general way, saying engineers should be careful without giving strong reasons. Sometimes it discusses civilians and damage, but the examples are mixed up and not very clear. Overall, the paper seems to argue that drones are bad and also useful at the same time, but it does not clearly prove either point. The conclusion repeats earlier ideas and says engineers have responsibilities, which is already obvious."""
STUDENT_SUMMARY = """Scope & purpose: The paper examines the ethical, technical, and humanitarian dimensions of drone warfare as revealed by the Russia–Ukraine conflict. It compares two representative platforms (a low-cost loitering munition and a higher-end reusable tactical UAV), explains how engineering choices influence both military effectiveness and civilian risk, documents concrete humanitarian impacts observed in Ukrainian cities, and draws implications for engineers, professional codes, and policy.

Technical comparison (design trade-offs):

The report contrasts cheap kamikaze loitering munitions (exemplified by the Shahed-136 class) with larger, reusable tactical drones (exemplified by the Bayraktar TB2 class).

Cheap loitering munitions prioritize low unit cost and mass producibility, enabling salvo/swarm attacks that saturate defenses but sacrifice precision, reusability, and reliability.

Reusable tactical drones prioritize endurance, sensors, and precision weapons; they enable targeted strikes and ISR (intelligence, surveillance, reconnaissance) but are costlier and fewer in number and vulnerable to air defenses.

Key engineering trade-offs: cost vs. precision, autonomy vs. human control, and reliability vs. operational complexity.

Autonomy, control, and accountability:

The report highlights varying autonomy levels: preprogrammed loiter-and-strike behavior versus real-time human-in-the-loop control via datalink.

Increased autonomy (e.g., automated target recognition) raises responsibility and verification challenges: who is accountable when an algorithm misidentifies a target?

The authors stress transparency, verification, and human oversight as necessary to preserve traceable decision chains and comply with laws of armed conflict.

Engineers’ ethical responsibilities:

Professional codes (the report cites the NSPE Code as an example) obligate engineers to prioritize public safety, honesty in reporting, and to raise concerns or withdraw from work that foreseeably endangers civilians.

The paper argues engineers should bake in safeguards (fail-safes, no-fire geofencing, robust verification) and resist design choices that make misuse or indiscriminate harm likely.

Ethical practice includes anticipating downstream misuse and participating in policy/dialogue to shape norms and regulation.

Humanitarian and societal impacts (case evidence):

The report compiles documented incidents and monitoring statistics showing short-range drones became a leading cause of civilian casualties in some months, and that loitering munitions often cause indiscriminate harm when guidance or targeting is poor. The UN monitoring findings and several Reuters-reported strikes are used as examples.

Case studies discussed include large combined missile-and-drone strikes on Kyiv and multiple strikes on passenger buses in Kherson and Marhanets that resulted in civilian deaths and injuries.

Beyond direct casualties and infrastructure damage (schools, power plants, apartments), persistent night-time drone barrages inflict widespread psychological harm — sleep deprivation, increased PTSD risk, and community-level erosion of trust in institutions and technology.

Broader implications for warfare & policy:

Cheap, effective strike drones lower the operational and political costs of kinetic action, potentially lowering the threshold for using force.

Proliferation risk: success in one conflict accelerates global acquisition and doctrinal adoption, increasing the chance of drone-driven escalation elsewhere.

The paper advocates for stronger international norms or treaties to limit fully autonomous lethal swarms and stricter controls on kamikaze loitering munitions.

Conclusions & recommendations (practical takeaways):

Design safeguards: engineers should implement human-in-the-loop controls, robust identification/verification systems, geofencing/no-fire zones, and clear fail-safe behaviors.

Ethical practice: engineers must apply professional codes (prioritizing public welfare), perform rigorous testing, and report safety shortfalls honestly.

Policy engagement: technical experts should contribute to regulation, standard setting, and public debate to ensure weapons development aligns with humanitarian law.

Monitoring & accountability: increased transparency and audit logs for targeting decisions are recommended so misuse can be investigated and responsibility assigned.

Mitigation & humanitarian response: strengthen civil defense, early-warning, and mental-health support in affected areas while pursuing diplomatic and legal avenues to curb indiscriminate use.

Key findings (brief):

Low-cost loitering munitions enable mass attacks but increase civilian risk; precision reusable UAVs reduce collateral harm but are costlier and scarcer.

Autonomy amplifies moral and legal ambiguity unless accompanied by verifiable human oversight and design constraints.

Drone warfare in Ukraine has produced measurable civilian harm (casualties, infrastructure damage, and psychological trauma), underscoring the need for ethical engineering and international norms."""
STUDENT_SUMMARY = STUDENT_SUMMARY.strip()
print(f"✅  Student summary loaded  ({len(STUDENT_SUMMARY.split()):,} words).")


✅  Student summary loaded  (118 words).


## 📊 Step 5 — Evaluation Engine

Loads the sentence-embedding model and defines all **ten scoring functions**.  
Each function returns a score from **0 to 10**.

| Metric | Method | Ground Truth |
|---|---|---|
| **Correctness** | 40% cosine vs lecture + 35% cosine vs reference + 25% ROUGE F1 | **Hybrid** |
| **Relevance** | Cosine similarity vs lecture paragraphs | Lecture |
| **Coherence** | AI judge — logical flow and structure | Student only |
| **Completeness** | 50% key-point embedding coverage + 50% ROUGE recall vs reference | **Hybrid** |
| **Conciseness** | Gaussian score centered on reference length | Reference |
| **Terminology** | Coverage of top domain terms from lecture | Lecture |
| **Hallucination** | AI detects claims absent from the lecture | Lecture |
| **Missing Key Points** | Embedding coverage of 15 extracted lecture key points | **Lecture key points** |
| **Factual Accuracy** | AI verifies specific facts vs lecture | Lecture |
| **Critical Analysis** | AI judges reasoning depth and synthesis | Student only |


In [31]:
# ── Load sentence-embedding model ────────────────────────────────────────────
print("⏳  Loading sentence-embedding model (first run downloads ~80 MB)…")
EMBEDDER = SentenceTransformer("all-MiniLM-L6-v2")
print("✅  Embedding model ready.\n")


# ═════════════════════════════════════════════════════════════════════════════
#  1. CORRECTNESS  — HYBRID: lecture (40%) + reference summary (35%) + ROUGE (25%)
# ═════════════════════════════════════════════════════════════════════════════
def score_correctness(student: str, reference: str, lecture: str) -> float:
    """
    Hybrid correctness:
      40% — cosine similarity vs raw lecture  (anchored to source)
      35% — cosine similarity vs AI reference (style/structure signal)
      25% — ROUGE F1 vs AI reference          (lexical overlap)
    This way a weak reference summary only affects 60% of the score.
    """
    emb_s = EMBEDDER.encode(student,   convert_to_tensor=True)
    emb_r = EMBEDDER.encode(reference, convert_to_tensor=True)
    emb_l = EMBEDDER.encode(lecture,   convert_to_tensor=True)

    cos_ref     = float(util.cos_sim(emb_s, emb_r))
    cos_lecture = float(util.cos_sim(emb_s, emb_l))

    sc    = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    rs    = sc.score(reference, student)
    rouge = (rs["rouge1"].fmeasure + rs["rougeL"].fmeasure) / 2

    hybrid = 0.40 * cos_lecture + 0.35 * cos_ref + 0.25 * rouge
    return round(hybrid * 10, 2)


# ═════════════════════════════════════════════════════════════════════════════
#  2. RELEVANCE  — coverage of the lecture's main topics
# ═════════════════════════════════════════════════════════════════════════════
def score_relevance(student: str, lecture: str) -> float:
    chunks = [p.strip() for p in re.split(r"\n{2,}", lecture) if len(p.strip()) > 60]
    if not chunks:
        chunks = [lecture]

    emb_s = EMBEDDER.encode(student, convert_to_tensor=True)
    emb_c = EMBEDDER.encode(chunks,  convert_to_tensor=True)
    sims  = util.cos_sim(emb_s, emb_c)[0]

    score = 0.4 * float(sims.mean()) + 0.6 * float(sims.max())
    return round(score * 10, 2)


# ═════════════════════════════════════════════════════════════════════════════
#  3. COHERENCE  — logical flow & structure, judged by AI
# ═════════════════════════════════════════════════════════════════════════════
def score_coherence(student: str) -> tuple:
    prompt = (
        "Evaluate the COHERENCE of the student summary below on a scale of 0–10.\n\n"
        "Coherence criteria:\n"
        "  • Ideas flow logically from one sentence/paragraph to the next\n"
        "  • Clear transitions and connective phrases\n"
        "  • No contradictions or abrupt topic jumps\n"
        "  • Well-organised, easy to follow\n\n"
        f"Student Summary:\n\"\"\"\n{student}\n\"\"\"\n\n"
        'Respond ONLY with valid JSON: {"score": <number 0-10>, "reason": "<1-2 sentences>"}'
    )
    try:
        raw  = openai_chat(prompt, max_tokens=200)
        m    = re.search(r"\{.*\}", raw, re.DOTALL)
        data = json.loads(m.group()) if m else {}
        return round(float(data.get("score", 5)), 2), data.get("reason", "No reason provided.")
    except Exception as e:
        return 5.0, f"Evaluation error: {e}"


# ═════════════════════════════════════════════════════════════════════════════
#  4. COMPLETENESS  — HYBRID: key-point embedding (50%) + ROUGE recall (50%)
# ═════════════════════════════════════════════════════════════════════════════
def score_completeness(student: str, reference: str, key_points: list) -> float:
    """
    Hybrid completeness:
      50% — per-point embedding coverage of extracted lecture key points
             (each point checked individually → average hit rate)
      50% — ROUGE recall vs AI reference summary
    Key points are extracted directly from the lecture, so this metric
    is robust even when the reference summary is incomplete.
    """
    # ── Part A: key-point coverage (lecture-anchored) ────────────────────────
    if key_points:
        emb_s  = EMBEDDER.encode(student,     convert_to_tensor=True)
        emb_kp = EMBEDDER.encode(key_points,  convert_to_tensor=True)
        sims   = util.cos_sim(emb_s, emb_kp)[0]
        # A point is "covered" if similarity > 0.40 threshold
        covered = float((sims > 0.40).float().mean())
    else:
        covered = 0.5   # fallback

    # ── Part B: ROUGE recall vs reference ────────────────────────────────────
    sc = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rs = sc.score(reference, student)
    recall = (rs["rouge1"].recall + rs["rouge2"].recall + rs["rougeL"].recall) / 3

    hybrid = 0.50 * covered + 0.50 * recall
    return round(hybrid * 10, 2)


# ═════════════════════════════════════════════════════════════════════════════
#  5. CONCISENESS  — appropriate length and information density
# ═════════════════════════════════════════════════════════════════════════════
def score_conciseness(student: str, reference: str) -> float:
    ratio = len(student.split()) / max(len(reference.split()), 1)
    score = np.exp(-((ratio - 1.0) ** 2) / (2 * 0.55 ** 2)) * 10
    return round(float(np.clip(score, 0, 10)), 2)


# ═════════════════════════════════════════════════════════════════════════════
#  6. TERMINOLOGY  — domain-specific vocabulary coverage
# ═════════════════════════════════════════════════════════════════════════════
def score_terminology(student: str, lecture: str) -> float:
    stop = set(stopwords.words("english"))

    def top_terms(text: str, n: int = 40) -> set:
        tokens = [w.lower() for w in word_tokenize(text) if w.isalpha() and len(w) > 3]
        tokens = [w for w in tokens if w not in stop]
        return {w for w, _ in Counter(tokens).most_common(n)}

    lecture_terms = top_terms(lecture, n=40)
    student_terms = {w.lower() for w in word_tokenize(student) if w.isalpha()}

    if not lecture_terms:
        return 5.0
    return round(len(lecture_terms & student_terms) / len(lecture_terms) * 10, 2)


# ═════════════════════════════════════════════════════════════════════════════
#  7. HALLUCINATION DETECTION  — claims in student not supported by lecture
# ═════════════════════════════════════════════════════════════════════════════
def score_hallucination(student: str, lecture: str) -> tuple:
    prompt = (
        "You are evaluating a student summary against a source lecture.\n\n"
        "Identify any claims, facts, or statements in the STUDENT SUMMARY that are "
        "NOT supported by or directly contradict the LECTURE. These are hallucinations.\n\n"
        f"LECTURE:\n\"\"\"\n{lecture}\n\"\"\"\n\n"
        f"STUDENT SUMMARY:\n\"\"\"\n{student}\n\"\"\"\n\n"
        "Return ONLY valid JSON:\n"
        '{"score": <0-10 where 10=no hallucinations, 0=many false claims>, '
        '"hallucinations": ["<claim1>", "<claim2>"] or [] if none}'
    )
    try:
        raw  = openai_chat(prompt, max_tokens=500)
        m    = re.search(r"\{.*\}", raw, re.DOTALL)
        data = json.loads(m.group()) if m else {}
        score = round(float(data.get("score", 7)), 2)
        items = data.get("hallucinations", [])
        detail = ("✅ No unsupported claims detected." if not items
                  else "⚠️ Unsupported claims: " + "; ".join(items[:4]))
        return score, detail
    except Exception as e:
        return 7.0, f"Evaluation error: {e}"


# ═════════════════════════════════════════════════════════════════════════════
#  8. MISSING KEY POINTS  — checked against LECTURE KEY POINTS (not reference)
# ═════════════════════════════════════════════════════════════════════════════
def score_missing_points(student: str, key_points: list) -> tuple:
    """
    Checks each extracted lecture key point against the student summary
    using embedding similarity. Points with similarity < 0.38 are flagged
    as likely missing. Score = fraction of key points covered × 10.
    This is fully independent of the reference summary quality.
    """
    if not key_points:
        return 5.0, "No key points available for comparison."

    emb_s  = EMBEDDER.encode(student,    convert_to_tensor=True)
    emb_kp = EMBEDDER.encode(key_points, convert_to_tensor=True)
    sims   = util.cos_sim(emb_s, emb_kp)[0].tolist()

    THRESHOLD = 0.38
    missed = [key_points[i] for i, s in enumerate(sims) if s < THRESHOLD]
    covered_frac = 1.0 - len(missed) / len(key_points)
    score = round(covered_frac * 10, 2)

    if not missed:
        detail = "✅ All key lecture points appear to be covered."
    else:
        # Show up to 3 missed points, truncated for display
        snippets = [p[:80] + "…" if len(p) > 80 else p for p in missed[:3]]
        detail = f"❌ Likely missing ({len(missed)}/{len(key_points)} pts): " + "; ".join(snippets)

    return score, detail


# ═════════════════════════════════════════════════════════════════════════════
#  9. FACTUAL ACCURACY  — numbers, names, statistics verified against lecture
# ═════════════════════════════════════════════════════════════════════════════
def score_factual_accuracy(student: str, lecture: str) -> tuple:
    prompt = (
        "Check whether the specific facts in the STUDENT SUMMARY are accurate "
        "according to the LECTURE (e.g., numbers, names, dates, statistics, "
        "technical specifications, percentages).\n\n"
        f"LECTURE:\n\"\"\"\n{lecture}\n\"\"\"\n\n"
        f"STUDENT SUMMARY:\n\"\"\"\n{student}\n\"\"\"\n\n"
        "Return ONLY valid JSON:\n"
        '{"score": <0-10 where 10=all facts correct, 0=many factual errors>, '
        '"errors": ["<error1>", "<error2>"] or [] if no errors, '
        '"reason": "<1 sentence overall assessment>"}'
    )
    try:
        raw  = openai_chat(prompt, max_tokens=500)
        m    = re.search(r"\{.*\}", raw, re.DOTALL)
        data = json.loads(m.group()) if m else {}
        score  = round(float(data.get("score", 7)), 2)
        errors = data.get("errors", [])
        reason = data.get("reason", "")
        detail = reason if not errors else reason + "  Errors: " + "; ".join(errors[:3])
        return score, detail
    except Exception as e:
        return 7.0, f"Evaluation error: {e}"


# ═════════════════════════════════════════════════════════════════════════════
#  10. CRITICAL ANALYSIS DEPTH  — reasoning, trade-offs, synthesis
# ═════════════════════════════════════════════════════════════════════════════
def score_critical_analysis(student: str) -> tuple:
    prompt = (
        "Evaluate the CRITICAL ANALYSIS DEPTH of the student summary on a scale of 0–10.\n\n"
        "Criteria:\n"
        "  • Student explains WHY, not just WHAT (causes, implications, consequences)\n"
        "  • Student identifies tensions, trade-offs, or ethical dilemmas\n"
        "  • Student synthesises ideas rather than just listing facts\n"
        "  • Student shows understanding of broader significance\n\n"
        f"Student Summary:\n\"\"\"\n{student}\n\"\"\"\n\n"
        'Respond ONLY with valid JSON: {"score": <number 0-10>, "reason": "<1-2 sentences>"}'
    )
    try:
        raw  = openai_chat(prompt, max_tokens=200)
        m    = re.search(r"\{.*\}", raw, re.DOTALL)
        data = json.loads(m.group()) if m else {}
        return round(float(data.get("score", 5)), 2), data.get("reason", "No reason provided.")
    except Exception as e:
        return 5.0, f"Evaluation error: {e}"


print("✅  All 10 scoring functions defined (hybrid mode).")


⏳  Loading sentence-embedding model (first run downloads ~80 MB)…
✅  Embedding model ready.

✅  All 10 scoring functions defined (hybrid mode).


## 🚀 Step 6 — Run the Evaluator

Run this cell to evaluate the student summary and view the full results dashboard.


In [32]:
def run_evaluation(student: str, reference: str, lecture: str,
                   key_points: list) -> tuple:
    """Run all ten evaluation metrics and return (scores_dict, overall_score)."""
    print("⏳  Evaluating student summary (hybrid mode)…")

    correctness                        = score_correctness(student, reference, lecture)
    print("  ✔  Correctness  (hybrid: lecture + reference + ROUGE)")
    relevance                          = score_relevance(student, lecture)
    print("  ✔  Relevance")
    coherence,         coh_reason      = score_coherence(student)
    print("  ✔  Coherence")
    completeness                       = score_completeness(student, reference, key_points)
    print("  ✔  Completeness  (hybrid: key-point coverage + ROUGE recall)")
    conciseness                        = score_conciseness(student, reference)
    print("  ✔  Conciseness")
    terminology                        = score_terminology(student, lecture)
    print("  ✔  Terminology")
    hallucination,     hal_detail      = score_hallucination(student, lecture)
    print("  ✔  Hallucination Detection")
    missing_pts,       miss_detail     = score_missing_points(student, key_points)
    print("  ✔  Missing Key Points  (lecture key points — independent ground truth)")
    factual_acc,       fact_detail     = score_factual_accuracy(student, lecture)
    print("  ✔  Factual Accuracy")
    critical_anal,     crit_reason     = score_critical_analysis(student)
    print("  ✔  Critical Analysis\n")

    scores = {
        "Correctness":       (correctness,   "Hybrid: 40% vs lecture · 35% vs reference · 25% ROUGE"),
        "Relevance":         (relevance,     "Coverage of the lecture's key topics and ideas"),
        "Coherence":         (coherence,     coh_reason),
        "Completeness":      (completeness,  "Hybrid: 50% lecture key-point coverage · 50% ROUGE recall"),
        "Conciseness":       (conciseness,   "Length efficiency relative to the reference summary"),
        "Terminology":       (terminology,   "Use of domain-specific vocabulary from the lecture"),
        "Hallucination":     (hallucination, hal_detail),
        "Missing Key Pts":   (missing_pts,   miss_detail),
        "Factual Accuracy":  (factual_acc,   fact_detail),
        "Critical Analysis": (critical_anal, crit_reason),
    }

    overall = round(sum(v for v, _ in scores.values()) / len(scores), 2)
    return scores, overall


def render_results(scores: dict, overall: float,
                   student: str, reference: str,
                   key_points: list) -> None:
    """Render a styled HTML results dashboard."""

    def bar(score: float) -> str:
        pct   = score / 10 * 100
        color = ("#e74c3c" if score < 4 else
                 "#f39c12" if score < 7 else "#2ecc71")
        return (
            f'<div style="background:#ecf0f1;border-radius:6px;height:16px;width:100%;">'
            f'<div style="background:{color};width:{pct:.1f}%;height:16px;'
            f'border-radius:6px;transition:width .4s;"></div></div>'
        )

    rows = ""
    for metric, (val, desc) in scores.items():
        icon = "🟢" if val >= 7 else "🟡" if val >= 4 else "🔴"
        rows += (
            f"<tr style='border-bottom:1px solid #eee'>"
            f"<td style='padding:11px 14px;font-weight:600;color:#000'>{icon} {metric}</td>"
            f"<td style='padding:11px 14px;text-align:center;font-weight:700;"
            f"font-size:1.15em;color:#000'>{val:.1f}"
            f"<span style='font-size:0.65em;color:#555;font-weight:400'> /10</span></td>"
            f"<td style='padding:11px 14px;min-width:200px'>{bar(val)}</td>"
            f"<td style='padding:11px 14px;color:#000;font-size:0.85em'>{desc}</td>"
            f"</tr>"
        )

    ov_color = "#e74c3c" if overall < 4 else "#f39c12" if overall < 7 else "#2ecc71"
    sw  = len(student.split())
    rw  = len(reference.split())
    kp  = len(key_points)

    html = f"""
    <div style="font-family:'Segoe UI',Arial,sans-serif;max-width:900px;margin:24px 0">

      <h2 style="border-bottom:3px solid #3498db;padding-bottom:10px;
                 color:#2c3e50;margin-bottom:18px">
        📊 Summary Evaluation Results
        <span style="font-size:0.55em;font-weight:400;color:#7f8c8d;margin-left:10px">
          Hybrid Mode — Lecture + Reference Summary
        </span>
      </h2>

      <!-- Stats bar -->
      <div style="display:flex;gap:16px;margin-bottom:18px;flex-wrap:wrap">
        <div style="background:#eaf4fb;border-radius:8px;padding:10px 18px;flex:1;min-width:130px">
          <div style="font-size:.75em;color:#555;text-transform:uppercase;letter-spacing:.5px">Student words</div>
          <div style="font-size:1.4em;font-weight:700;color:#2c3e50">{sw:,}</div>
        </div>
        <div style="background:#eafaf1;border-radius:8px;padding:10px 18px;flex:1;min-width:130px">
          <div style="font-size:.75em;color:#555;text-transform:uppercase;letter-spacing:.5px">Reference words</div>
          <div style="font-size:1.4em;font-weight:700;color:#2c3e50">{rw:,}</div>
        </div>
        <div style="background:#fef9e7;border-radius:8px;padding:10px 18px;flex:1;min-width:130px">
          <div style="font-size:.75em;color:#555;text-transform:uppercase;letter-spacing:.5px">Length ratio</div>
          <div style="font-size:1.4em;font-weight:700;color:#2c3e50">{sw/max(rw,1):.2f}×</div>
        </div>
        <div style="background:#fdebd0;border-radius:8px;padding:10px 18px;flex:1;min-width:130px">
          <div style="font-size:.75em;color:#555;text-transform:uppercase;letter-spacing:.5px">Key points</div>
          <div style="font-size:1.4em;font-weight:700;color:#2c3e50">{kp}</div>
        </div>
        <div style="background:#f5eef8;border-radius:8px;padding:10px 18px;flex:1;min-width:130px">
          <div style="font-size:.75em;color:#555;text-transform:uppercase;letter-spacing:.5px">Metrics</div>
          <div style="font-size:1.4em;font-weight:700;color:#2c3e50">{len(scores)}</div>
        </div>
      </div>

      <!-- Scores table -->
      <table style="width:100%;border-collapse:collapse;background:#fff;
                    border-radius:10px;overflow:hidden;
                    box-shadow:0 2px 10px rgba(0,0,0,.08)">
        <thead>
          <tr style="background:#2c3e50;color:#fff">
            <th style="padding:13px 14px;text-align:left;width:17%">Metric</th>
            <th style="padding:13px 14px;width:9%">Score</th>
            <th style="padding:13px 14px;text-align:left;width:24%">Visual</th>
            <th style="padding:13px 14px;text-align:left">Details / AI Feedback</th>
          </tr>
        </thead>
        <tbody style="font-size:.9em">{rows}</tbody>
      </table>

      <!-- Overall score -->
      <div style="margin-top:20px;padding:16px 22px;
                  background:#f8f9fa;border-left:6px solid {ov_color};
                  border-radius:5px;display:flex;align-items:center;gap:16px">
        <span style="font-size:1em;font-weight:600;color:#2c3e50">Overall Score</span>
        <span style="font-size:2em;font-weight:800;color:{ov_color}">{overall:.2f}</span>
        <span style="font-size:1em;color:#888">/ 10</span>
      </div>

      <p style="margin-top:12px;font-size:.78em;color:#666">
        🟢 Good (7–10) &nbsp;|&nbsp; 🟡 Needs Improvement (4–7)
        &nbsp;|&nbsp; 🔴 Poor (0–4) &nbsp;·&nbsp;
        <strong>Hybrid mode:</strong> Correctness &amp; Completeness are anchored to the
        original lecture — robust against a weak reference summary.
      </p>
    </div>
    """
    display(HTML(html))


# ── Run ───────────────────────────────────────────────────────────────────────
scores, overall = run_evaluation(
    STUDENT_SUMMARY, REFERENCE_SUMMARY, LECTURE, LECTURE_KEY_POINTS
)
render_results(scores, overall, STUDENT_SUMMARY, REFERENCE_SUMMARY, LECTURE_KEY_POINTS)


⏳  Evaluating student summary (hybrid mode)…
  ✔  Correctness  (hybrid: lecture + reference + ROUGE)
  ✔  Relevance
  ✔  Coherence
  ✔  Completeness  (hybrid: key-point coverage + ROUGE recall)
  ✔  Conciseness
  ✔  Terminology
  ✔  Hallucination Detection
  ✔  Missing Key Points  (lecture key points — independent ground truth)
  ✔  Factual Accuracy
  ✔  Critical Analysis



Metric,Score,Visual,Details / AI Feedback
🟡 Correctness,6.0 /10,,Hybrid: 40% vs lecture · 35% vs reference · 25% ROUGE
🟢 Relevance,7.6 /10,,Coverage of the lecture's key topics and ideas
🟡 Coherence,4.0 /10,,"The summary lacks clear organization and logical flow, with abrupt topic jumps and mixed examples that hinder understanding. Additionally, the contradictory stance on the usefulness of drones detracts from coherence."
🟡 Completeness,4.2 /10,,Hybrid: 50% lecture key-point coverage · 50% ROUGE recall
🔴 Conciseness,3.4 /10,,Length efficiency relative to the reference summary
🔴 Terminology,1.2 /10,,Use of domain-specific vocabulary from the lecture
🔴 Hallucination,2.0 /10,,"⚠️ Unsupported claims: The student summary claims that the paper does not explain the Iranian and Turkish drones clearly, while the lecture provides detailed comparisons of the Shahed-136 and Bayraktar TB2.; The summary states that the paper discusses ethics only in a general way without giving strong reasons, whereas the lecture elaborates on specific ethical dilemmas and responsibilities of engineers.; The student summary suggests that the examples of civilian damage are mixed up and not very clear, while the lecture provides specific instances and statistics regarding civilian casualties from drone strikes.; The summary claims the paper argues that drones are bad and useful at the same time without proving either point, while the lecture presents a nuanced discussion of the ethical implications and responsibilities associated with drone warfare."
🟢 Missing Key Pts,8.0 /10,,"❌ Likely missing (3/15 pts): The Shahed-136 has a production cost of tens of thousands of USD, allowing for m…; The Shahed-136's design prioritizes cost-effectiveness, enabling swarming tactic…; The Bayraktar TB2 can carry precision weapons and has a flight endurance of 27 h…"
🔴 Factual Accuracy,3.0 /10,,"The summary lacks accuracy and detail, misrepresenting key facts and ethical considerations from the lecture. Errors: The summary does not accurately represent the specific details about the Shahed-136 and Bayraktar TB2 drones, such as their specifications, costs, and operational capabilities.; The summary fails to mention the significant humanitarian impact of drone strikes, including specific statistics and examples provided in the lecture.; The ethical implications discussed in the lecture are oversimplified in the summary, lacking the depth and specific references to engineering codes and responsibilities."
🔴 Critical Analysis,3.0 /10,,"The summary identifies some ethical considerations and mentions the dual nature of drones, but it lacks depth in explaining causes, implications, and broader significance. Additionally, it does not effectively synthesize ideas or clarify the tensions and trade-offs involved."
